In [19]:
import pandas as pd

In [20]:
# Daily processing
daily_folder = r"Z:\PhD_Datasets&Analysis\Info_Inputs\Streamflow_Stations\Climate_Sensitive_Stations-GRDC\2025-02-13_17-18_Daily"
# Check if the exported file can be read correctly
df_cleaned = pd.read_csv(daily_folder + "\_DataFrames\Joined_Daily_Sts_DFs.csv", index_col="YYYY-MM-DD")

# Reading file content
with open(daily_folder + "//baseflow//file.lst", "r") as file:
    content = file.readlines()

position = 6
space = "     " 
header = content[:position] # Take the header of the baseflow master file
lines_to_write = []

stations = df_cleaned.columns.to_list()

for id_station in stations:

    print("Processing station " + id_station)
 
    # Setting up files for the following baseflow calculation
    temp_df2 = df_cleaned[id_station].reset_index()
    temp_df2 = temp_df2[temp_df2[id_station].notnull()] # Remove rows that have NaN values in the specified column only. 
    temp_df2 = temp_df2.rename(columns={"YYYY-MM-DD":"Date", id_station:"FlowValue"}) # Rename columns to fit the required format for baseflow calculation
    temp_df2["Date"] = temp_df2["Date"].apply(lambda row:  row.replace("-","")) # There must be no spaces in the date portion (YYYYMMDD) of the line
    temp_df2 = temp_df2.set_index("Date")
    temp_df2.to_csv(daily_folder + "//baseflow//" + id_station + ".csv", sep=" ") # Save file using space delimited format for baseflow calculations

    lines_to_write.append(id_station + ".csv" + space + id_station + ".out\n")

new_content = header + lines_to_write

# Writing modified content back
with open(daily_folder + "//baseflow//file.lst", "w") as file:
    file.write(''.join(new_content))

Processing station 1257100
Processing station 1309620
Processing station 1769100
Processing station 2106100
Processing station 2178200
Processing station 2178951
Processing station 2178960
Processing station 2180400
Processing station 2180600
Processing station 2180711
Processing station 2180712
Processing station 2181100
Processing station 2181200
Processing station 2181300
Processing station 2181500
Processing station 2182050
Processing station 2182150
Processing station 2182250
Processing station 2517500
Processing station 2517550
Processing station 2517600
Processing station 2517920
Processing station 3613150
Processing station 3617110
Processing station 3617120
Processing station 3617810
Processing station 3617811
Processing station 3617812
Processing station 3617813
Processing station 3617814
Processing station 3617820
Processing station 3618051
Processing station 3618052
Processing station 3618053
Processing station 3618090
Processing station 3618110
Processing station 3618115
P